## Convolutional Neural Network Model with Max Pooling

In [3]:
'''
Step 1: Loading MNIST Train Dataset
'''

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
import torchvision.datasets as dsets

# download train dataset

train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform = transforms.ToTensor(),
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                          train=False,
                          transform=transforms.ToTensor()) # download not required, already downloaded in last step


'''
Step 2: Make Dataset Iterable
'''

batch_size = 100
n_iters = 3000
num_epochs = n_iters / (len(train_dataset) / batch_size)
num_epochs = int(num_epochs) 
num_epochs

train_loader = torch.utils.data.DataLoader(dataset = train_dataset,
                                          batch_size = batch_size,
                                          shuffle = True)

test_loader = torch.utils.data.DataLoader(dataset = test_dataset,
                                         batch_size = batch_size,
                                         shuffle = False)


'''
Step 3: Create Model Class
'''

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels = 1, out_channels = 16, kernel_size=5, stride=1, padding=2)
        self.relu1 = nn.ReLU()
        
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
        
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels = 16, out_channels =32, kernel_size = 5, stride = 1, padding = 2)
        self.relu2 = nn.ReLU()
        
        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)
        
        # Fully connected 1(read out)
        self.fc1 = nn.Linear(32*7*7, 10)
        
    def forward(self,x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2
        out = self.cnn2(out)
        out = self.relu2(out)
        
        # Max pool 2
        out = self.maxpool2(out)
        
        #Resize 
        # original size : (100, 32, 7, 7)
        # out.size(0): 100
        # New out size : (100, 32*7*7)
        
        out = out.view(out.size(0), -1)
        
        #Linear function (read out)
        out = self.fc1(out)

        return out

    
'''
Step 4: Instantiate Model Class
'''



model = CNNModel()

'''
Step 5: Instantiate Loss Class
'''

criterion = nn.CrossEntropyLoss()

'''
Step 6: Instantiate Optimizer Class
'''

learning_rate = 0.01

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


'''
Step 7: Train Model
'''

iter = 0

for epoch in range(num_epochs):
    for i,(images, labels) in enumerate(train_loader):
        # load images as varible
        images = Variable(images)
        labels = Variable(labels)
        
        # clear gradients w.r.t parameters
        optimizer.zero_grad()
        
        # Forward pass to get output
        outputs = model(images)
        
        # Calculate Loss : softmax --> cross entropy loss
        loss = criterion(outputs, labels)
        
        # getting gradients w.r.t parameters
        loss.backward()
        
        # updating parameters
        optimizer.step()
        
        iter += 1
        
        if iter % 500 == 0:
            # calculate Accuracy
            correct = 0
            total = 0
            #iterate through test dataset
            for images, labels in test_loader:
                #load images to a Torch variable
                images = Variable(images)
                
                # Forward pass only to get outputs
                outputs = model(images)
                
                # get predictions from the maximum value
                _, predicted = torch.max(outputs.data, 1)
                
                # Total number of labels 
                total += labels.size(0)
                
                # Total correct predictions
                correct += (predicted == labels).sum()
            
            accuracy = float(100 * correct / total)
            
            print('Iteration: {}. Loss: {}. Accuracy: {}'.format(iter, loss.data, accuracy))



Iteration: 500. Loss: 0.4569580852985382. Accuracy: 89.0
Iteration: 1000. Loss: 0.1746843010187149. Accuracy: 93.0
Iteration: 1500. Loss: 0.20421817898750305. Accuracy: 94.0
Iteration: 2000. Loss: 0.16929049789905548. Accuracy: 95.0
Iteration: 2500. Loss: 0.12796223163604736. Accuracy: 96.0
Iteration: 3000. Loss: 0.05112951621413231. Accuracy: 97.0
